# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [180]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [181]:
city_data=pd.read_csv("cityweather_Output.csv",index_col=False)
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,torbay,47.6666,-52.7314,15.16,97,100,5.66,CA,1663021246
1,barrow,71.2906,-156.7887,3.01,93,100,8.75,US,1663021246
2,grand gaube,-20.0064,57.6608,20.77,70,66,1.34,MU,1663021246
3,beloha,-25.1667,45.0500,17.51,97,7,1.72,MG,1663021247
4,jamestown,42.0970,-79.2353,20.85,71,20,3.60,US,1663021247


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [182]:
locations=city_data[["Lat","Lng"]].astype(float)


In [183]:
gmaps.configure(api_key=g_key)

In [184]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=city_data["Humidity"].astype(float), 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 6)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [185]:
city_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        555 non-null    object 
 1   Lat         555 non-null    float64
 2   Lng         555 non-null    float64
 3   Max Temp    555 non-null    float64
 4   Humidity    555 non-null    int64  
 5   Cloudiness  555 non-null    int64  
 6   Wind Speed  555 non-null    float64
 7   Country     553 non-null    object 
 8   Date        555 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 39.1+ KB


In [186]:
vacation_city=city_data.loc[(city_data["Max Temp"]<27)&(city_data["Max Temp"]>21)]

In [187]:
vacation_city=city_data.loc[(city_data["Max Temp"]<27) 
                           & (city_data["Max Temp"]>21) 
                           & (city_data["Wind Speed"] <4.5)
                            & (city_data["Cloudiness"]==0)]
vacation_city.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,gat,31.6100,34.7642,25.38,100,0,0.35,IL,1663021310
254,tucuman,-26.8241,-65.2226,22.00,17,0,2.57,AR,1663021343
346,gaeta,41.2183,13.5628,21.52,65,0,2.47,IT,1663021374
355,santiago del estero,-27.7951,-64.2615,22.23,17,0,2.91,AR,1663021377
518,tsumeb,-19.2333,17.7167,23.19,13,0,4.43,NaN,1663021203


In [188]:
vacation_city.shape

(7, 9)

In [189]:
vacation_clean_df=vacation_city.dropna()

In [190]:
vacation_clean_df.shape


(6, 9)

In [191]:
vacation_clean_df.head(6)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,gat,31.6100,34.7642,25.38,100,0,0.35,IL,1663021310
254,tucuman,-26.8241,-65.2226,22.00,17,0,2.57,AR,1663021343
346,gaeta,41.2183,13.5628,21.52,65,0,2.47,IT,1663021374
355,santiago del estero,-27.7951,-64.2615,22.23,17,0,2.91,AR,1663021377
525,alamosa,37.4695,-105.8700,26.99,12,0,2.57,US,1663021438
544,port said,31.2565,32.2841,26.97,72,0,3.03,EG,1663021446


In [192]:
vacation_clean_df.reset_index(drop=True, inplace=True)


In [193]:
vacation_clean_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [194]:
vacation_clean_df["Hotel Name"] =""
vacation_clean_df
    

C:\Users\ashle\AppData\Local\Temp\ipykernel_17664\1424943650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_clean_df["Hotel Name"] =""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,gat,31.6100,34.7642,25.38,100,0,0.35,IL,1663021310,
1,tucuman,-26.8241,-65.2226,22.00,17,0,2.57,AR,1663021343,
2,gaeta,41.2183,13.5628,21.52,65,0,2.47,IT,1663021374,
3,santiago del estero,-27.7951,-64.2615,22.23,17,0,2.91,AR,1663021377,
4,alamosa,37.4695,-105.8700,26.99,12,0,2.57,US,1663021438,
5,port said,31.2565,32.2841,26.97,72,0,3.03,EG,1663021446,


In [195]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [196]:

# Use the lat/lng we recovered to identify hotels
import json


for index, row in vacation_clean_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url and convert to json
    name_hotel = requests.get(base_url, params=params).json()
    #print(name_hotel)
    #print(json.dumps(name_hotel, indent=4, sort_keys=True))
    
    try:
        vacation_clean_df.loc[index,"Hotel Name"]=name_hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
vacation_clean_df    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,gat,31.6100,34.7642,25.38,100,0,0.35,IL,1663021310,OlusHome
1,tucuman,-26.8241,-65.2226,22.00,17,0,2.57,AR,1663021343,Dallas Hotel Tucuman
2,gaeta,41.2183,13.5628,21.52,65,0,2.47,IT,1663021374,Hotel Serapo
3,santiago del estero,-27.7951,-64.2615,22.23,17,0,2.91,AR,1663021377,Apartamentos Castelli 90
4,alamosa,37.4695,-105.8700,26.99,12,0,2.57,US,1663021438,Best Western Alamosa Inn
5,port said,31.2565,32.2841,26.97,72,0,3.03,EG,1663021446,Aracan Hotel


In [197]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_clean_df.iterrows()]
locations = vacation_clean_df[["Lat", "Lng"]]

In [198]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))